In [1]:
from engine import Tensor
import nn as nn
import functional as F
from optim import SGD
import numpy as np

In [2]:
# use torchvision to get mnist data
import torch
import torchvision 
from torchvision import transforms

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MLP definition

In [3]:
class MLP(nn.Module):
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = []
        for i in range(len(nouts)):
            self.layers.append(nn.Linear(sz[i], sz[i+1]))
            if i != len(nouts) - 1:
                self.layers.append(nn.Sigmoid())
    
    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

Training

In [4]:
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.Compose([
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download=True)

9913344it [00:00, 10173210.18it/s]                                              


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



29696it [00:00, 4100544.91it/s]                                                 

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



1649664it [00:00, 11950266.43it/s]                                              


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



5120it [00:00, 3292676.55it/s]                                                  


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
# put training data in lists for preprocessing
xs = []
ys = []

for x, y in train_dataset:
    xs.append(x)
    ys.append(y)

len(xs), len(ys)

(60000, 60000)

In [10]:
# convert Pytorch tensors to my Tensors
xs = list(map(torch.flatten, xs))
xs = list(map(lambda x: x.numpy(), xs))
xs = list(map(Tensor, xs))

In [11]:
model = MLP(784, [16, 16, 10])
optimizer = SGD(model.parameters(), lr=0.1)

# training loop
max_epochs = 3
for epoch in range(max_epochs):
    for it, (x, y) in enumerate(zip(xs, ys)):
        # forward pass
        ypred = model(x)
        loss = F.cross_entropy(ypred, y)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # update
        optimizer.step()
        
        if it % 10000 == 0:
            print(f"epoch: {epoch}, it: {it}, loss: {loss}")

NameError: name 'MLP' is not defined

Testing

In [8]:
test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.Compose([
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                          download=True)

In [9]:
# put test data in lists for preprocessing
test_xs = []
test_ys = []

for x, y in test_dataset:
    test_xs.append(x)
    test_ys.append(y)

len(test_xs), len(test_ys)


(10000, 10000)

In [10]:
# convert PyTorch tensors to my Tensor
test_xs = list(map(torch.flatten, test_xs))
test_xs = list(map(lambda x: x.numpy(), test_xs))
test_xs = list(map(Tensor, test_xs))

In [11]:
# evaluate accuracy on test set
correct = 0
total = 0
for x, y in zip(test_xs, test_ys):
    logits = model(x)
    pred = np.argmax(logits.data)
    correct += (y == pred)
    total += 1

print(f"accuracy: {correct / total}")

accuracy: 0.8861
